In [ ]:

!pip install datasets

from datasets import load_dataset

dataset = load_dataset("SKNahin/bengali-transliteration-data")

dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 5006
    })
})

In [ ]:
from datasets import load_dataset
dataset = load_dataset("SKNahin/bengali-transliteration-data")

# Splitting the dataset (80/20 split)
train_test_split = dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']


In [ ]:
val_dataset

Dataset({
    features: ['bn', 'rm'],
    num_rows: 1002
})

In [ ]:
print(dataset['train'][0:5])

{'bn': ['স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???', 'ও গুলা টরেন্ট সাইট এ পাবেন', 'ভক্কর চক্কর পোস্ট একটা করলেই এপ্রুভড.… নিশ্চই  ঘাবলা আছে', 'আমি টেস্ট করেই কোড দিছি…', 'এতো কষ্টের কি আছে সাকিবওয়াপ.টক,সাকিবওয়াপ.মল&এআইওভিডিওড্ল.মল থেকে সহজেই ডাউনলোড করা যায়'], 'rm': ['scroll kore 20/30 second er video pann nai???', 'o gula Torrent site e paben', 'vokkor chokkor post akta korlei approved…. nishchoi ghabla ache', 'ami test koreii code disi…', 'eto koster ki ache shakibwap.tk,shakibwap.ml&aiovideodl.ml theke shohojei downlod kora jay']}


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi") # Example tokenizer

def tokenize_function(examples):
    inputs = examples["bn"]
    targets = examples["rm"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/4004 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

In [ ]:
def filter_function(example):
    return len(example["bn"].split()) > 3 and len(example["rm"].split()) > 3

train_dataset = train_dataset.filter(filter_function)
val_dataset = val_dataset.filter(filter_function)


Filter:   0%|          | 0/4004 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1002 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    run_name="my_banglish_to_bangla_run",
    report_to=[],
    fp16=True  # Enable mixed precision training
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer  # Changed from tokenizer to processing_class
)

trainer.train()
trainer.save_model()

Epoch,Training Loss,Validation Loss
